In [2]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [4]:
%run -i ../../zkstats/core.py

In [5]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [7]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())
x1_vals = data['x1']
x2_vals = data['x2']
y_vals = data['y']

createDummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())
dummy_x1_vals = dummy_data['x1']
dummy_x2_vals = dummy_data['x2']
dummy_y_vals = dummy_data['y']


def stacked_x(*args):
    result = np.column_stack((*args, np.ones_like(args[0])))
    return result

x_one = stacked_x(x1_vals, x2_vals)
dummy_x_one = stacked_x(dummy_x1_vals, dummy_x2_vals)

w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_one.transpose(), x_one)), x_one.transpose()), y_vals)
dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(dummy_x_one.transpose(), dummy_x_one)), dummy_x_one.transpose()), dummy_y_vals)

print("reg fit: ", w_vals)
w_tensor = torch.tensor(w_vals).reshape(1,-1,1)
dummy_w_tensor = torch.tensor(dummy_w_vals).reshape(1,-1,1)


reg fit:  [1.98835287 3.14238058 0.06413647]


In [9]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w - X_T @ Y)) <= 0.01 * torch.sum(torch.abs(X_T @ Y)),
            self.w
        )
    

verifier_define_calculation(dummy_data_path, ['x1','x2','y'],sel_dummy_data_path,verifier_model, verifier_model_path)

In [10]:
# prover calculates settings, send to verifier

theory_output = w_tensor
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0])), dim=2)
        X_T = torch.transpose(X_one, 1, 2)
        return (
            torch.sum(torch.abs(X_T @ X_one @ self.w - X_T @ Y)) <= 0.01 * torch.sum(torch.abs(X_T @ Y)),
            self.w
        )
# try scale = [3] for more accuracy
prover_gen_settings(data_path,['x1','x2','y'], sel_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

Theory output:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-23296,18300],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":11808,"total_assignments":794,"total_const_size":13,"model_instance_shapes":[[1],[1,3,1]],"model_output_scales":[0,0],"model_input_scales":[0,0,0],"module_sizes":{"kzg":[],"poseidon":[11808,[3]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [11]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 6.932987928390503 seconds
Theory output:  tensor([[[1.9884],
         [3.1424],
         [0.0641]]], dtype=torch.float64)
!@# compiled_model exists? False
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 2.0
witness result 2 : 3.0
witness result 3 : 0.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [0, 0, 0, 0]]], 'proof': '10363b8402c70e7b80d4f2acd6c3babe01485159a5bccda49d49459ca566e85e2d2c53f12010849171586e24e4d7162f7d183277453197b328311743f255dcb2162cacf200e8d9766319ccc0f01761dc05e8d247ab9390cd3207fd13399d1bb906e3ff51f966b2ed8bffc54e226603bd1c18835dae832c7fb6ed3d8af340a18f28aa38b6ac3145e9c0d93d21d2446d7ee7890d4a1e590322e9406b6e5d823c86258bbb8b7c7ea512419fa7fcd6d57e52285e6ef29905fb822bf0d1648e4842fd0362fd965f28966b3bd0404e74de92e4175bbee869f497d5

In [12]:
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  3
prf instances:  [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [0, 0, 0, 0]]]
proof boolean:  1.0
proof result 1 : 2.0
proof result 2 : 3.0
proof result 3 : 0.0
verified
